In [ ]:
import requests
from pprint import pprint,pp
def pprint_response(r):
    print(r.status_code)
    pp(dict(r.headers))
    print()
    pp(r.json())
    print(len(r.json()))
API_KEY="e967b89f3f884bccea030b81afba466eb4008a031255dfd9b3aa2d9aa0d03739"
u1 = {'name': 'Eva', 'email': 'eva@gmail.com', 'gender': 'female', 'status': 'inactive'}
u2 = {'name': 'Ana', 'email': 'ana2@gmail.com', 'gender': 'female', 'status': 'active'}
u3 = {'name': 'Pepe', 'email': 'pepe@gmail.com', 'gender': 'female', 'status': 'inactive'}
u4= {'name':'Pedro','email':'pedroemail@gmail.com','gender':'male','status':'active'}
URL="https://gorest.co.in/"
response=requests.get("https://gorest.co.in//public/v2/users",headers={'Authorization':f'Bearer {API_KEY}'})
pp(response.json())
def inserta_usuarios(datos, token):
    """ Inserta todos los usuarios de la lista y devuelve True si todos han sido insertados correctamente """
    url=URL+"/public/v2/users"
    contador=0
    for user in datos:
        print(user)
        response=requests.post(url,headers={'Authorization':f'Bearer {token}'},
                            data=user)
        print(response.status_code)
        if response.status_code==201:
            contador+=1
    return contador
    
print(inserta_usuarios([u1,u2,u3,u4],API_KEY))

In [ ]:
def get_ident_email(email, token):
    """ Devuelve el identificador del usuario cuyo email sea *exactamente* el pasado como parámetro.
        En caso de que ese usuario no exista devuelve None """
    url=URL+"/public/v2/users"
    response=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    users=response.json()
    for user in users:
        if user["email"]==email:
            return user["id"]
    return None

print(get_ident_email("pedroemail@gmail.com",API_KEY))
print(get_ident_email("Buenos_dias@gmail.com",API_KEY))

In [ ]:
def borra_usuario(email, token):
    """ Elimina el usuario cuyo email sea *exactamente* el pasado como parámetro. En caso de éxito en el
        borrado devuelve True. Si no existe tal usuario devolverá False """
    id=get_ident_email(email,token)
    if(id is None):
        return False
    url=f'{URL}/public/v2/users/{id}'
    response=requests.delete(url,headers={'Authorization':f'Bearer {token}'})
    return response.status_code==204
print(borra_usuario("ana2@gmail.com",API_KEY))

In [36]:
#suponiendo que no se pueden añadir dos tareas con el mismo nombre
def inserta_todo__mal(email, token, title, due_on, status='pending'):
    """ Inserta un nuevo ToDo para el usuario con email exactamente igual al pasado. Si el ToDo ha sido insertado
        con éxito devolverá True, en otro caso devolverá False """
    id=get_ident_email(email,token)
    if(id is None):
        return False
    url=f'{URL}/public/v2/users/{id}'
    response=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    user=response.json()
    lista_tareas= user.get("tareas",[])
    print (lista_tareas)
    lista_tareas.append({"due_on":due_on,"status":status,"title":title})
    print (lista_tareas)
    response2=requests.patch(url,headers={'Authorization':f'Bearer {token}'},
                            data={"tareas":lista_tareas})
    pprint_response(response2)
    r=requests.get(url,headers={'Authorization':f'Bearer {token}'})
    print(r.json())
#inserta_todo("ana@gmail.com",API_KEY,"Todo 2","2034-07-28 11:3",'pending')

In [ ]:
def inserta_todo(email, token, title, due_on, status='pending'):
    """ Inserta un nuevo ToDo para el usuario con email exactamente igual al pasado. Si el ToDo ha sido insertado
        con éxito devolverá True, en otro caso devolverá False """
    
    user_id = get_ident_email(email, token)

    if user_id is None:
        return False

    response = requests.post(f'https://gorest.co.in/public/v2/users/{user_id}/todos',
                            headers={'Authorization': f'Bearer {token}'},
                            data = {'title': title,
                                    'due_on': due_on,
                                    'status': status}
                            )
    
    return response.status_code == 201

print(inserta_todo("pedroemail@gmail.com",API_KEY,"todo1","2034-07-28 11:30","pending"))
print(inserta_todo("pedroemail@gmail.com",API_KEY,"todo2","2034-07-30 11:35","pending"))
print(inserta_todo("pedroemail@gmail.com",API_KEY,"todo3","2034-08-15 11:37","pending"))
print(inserta_todo("pedroemail@gmail.com",API_KEY,"todo4","2022-07-28 11:26","pending"))

In [ ]:
def lista_todos(email, token):
    """ Devuelve una lista de diccionarios con todos los ToDo asociados al usuario con el email pasado como
        parámetro """
    
    user_id = get_ident_email(email, token)

    if user_id is None:
        return []
    
    response = requests.get(f'https://gorest.co.in/public/v2/users/{user_id}/todos',
                            headers={'Authorization': f'Bearer {token}'})
    
    if response.status_code == 200:
        return response.json()
    return []
    
pp(lista_todos("pedroemail@gmail.com",API_KEY))

In [ ]:
from datetime import datetime

def lista_todos_no_cumplidos(email, token):
    """ Devuelve una lista de diccionarios con todos los ToDo asociados al usuario con el email pasado como
        parámetro que están pendientes (status=pending) y cuya fecha tope (due_on) es anterior a la fecha
        y hora actual. Para comparar las fechas solo hay que tener en cuenta el dia, la hora y los minutos; es
        decir, ignorar los segundos, microsegundos y el uso horario """
    
    user_id = get_ident_email(email, token)

    if user_id is None:
        return []
    
    todos = lista_todos(email, token)
    
    ahora = datetime.now().replace(second=0, microsecond=0)
    
    tareas_no_cumplidas = []
    for tarea in todos:
        if tarea['status'] == 'pending':

            fecha_due = datetime.strptime(tarea['due_on'][:16], '%Y-%m-%dT%H:%M')
            
            if fecha_due > ahora:
                tareas_no_cumplidas.append(tarea)
    
    return tareas_no_cumplidas

print(lista_todos_no_cumplidos("pedroemail@gmail.com",API_KEY))